#NOTEBOOK STEAM ANALYSIS (PYSPARK)

<img src="https://th.bing.com/th/id/OIG.2dAGDtq6kLNfFrMJgA40?pid=ImgGn" alt="Image" width="30%" height="30%">

## IMPORTATION DES LIBRAIRIES ET DU FICHIER

In [0]:
# Importation des librairies
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, BooleanType, DateType, DecimalType
spark

In [0]:
# Importation du fichier Json

filepath = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"
df = spark.read.json(filepath)
df_count = df.count()
df.show(5)

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows



In [0]:
# Affichage du schéma

df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

## SOMMAIRE

<h5>PARTIE 1 : ANALYSE MACRO</h5>

A) Editeurs ayant le plus publié

B) Les jeux les mieux notés

C) Evolution des sorties dans le temps

D) Distrubtion des prix de vente

E) Les jeux en promotion

F) Les langues les plus représentées

G) Les jeux réservés aux personnes de 16 ans et plus


<h5>PARTIE 2 : ANALYSE PAR CATEGORIE</h5>

A) Les catégories les plus représentées

B) Les catégories les mieux notés

C) Les catégories préférés des éditeurs

D) Les catégories les plus rentables

<h5>PARTIE 3 : ANALYSE PAR PLATEFORME</h5>

A) Les disponiblités des jeux selon les systèmes d'exploitation

B) L'influence des catégories sur le choix des plateformes

## PARTIE 1 : ANALYSE MACRO

### A) Editeurs ayant le plus publié

In [0]:
df_publisher = df.groupBy('data.publisher')\
    .count()\
    .sort('count', ascending=False)\
    .filter(F.col('publisher') != '')\
    .limit(10)

display(df_publisher)

publisher,count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141
Choice of Games,140
Sekai Project,132
HH-Games,132
Ubisoft,127
Laush Studio,126


Databricks visualization. Run in Databricks to view.

> Editeurs ayant le plus publié

Si on se réfère au top 10 des éditeurs ayant le plus grand nombre de jeux publiés, Big Fish Games est l'éditeur le plus prolifique sur la plateforme Steam, avec un total de 422 jeux publiés.

Il est loin devant l'éditeur 8floor qui se place à la seconde place avec 202 jeux.

À partir de la troisième place, les éditeurs, tels que SEGA, Square Enix, et Ubisoft, sont au coude à coude en termes de nombre de jeux publiés, avec des chiffres relativement proches, variant entre 126 et 165 jeux. Cette compétition serrée souligne l'importante contribution de plusieurs éditeurs majeurs à la diversité de la bibliothèque de jeux sur la plateforme Steam.

On va conserver ce top 10 des éditeurs, ce qui nous permettra d'analyser leurs catégories de jeux préférées.

### B) Les jeux les mieux notés

In [0]:
# Filtre avec un nombre de votes > 500.000

df_games = df.select('data.name', 'data.publisher', 'data.positive', 'data.negative')\
    .withColumn('total_reviews', F.col('positive') + F.col('negative'))\
    .withColumn('positive_ratio', F.col('positive') / F.col('total_reviews'))\
    .filter(F.col('total_reviews') >= 250000)\
    .orderBy('positive_ratio', ascending=False)\
    .limit(10)

display(df_games)

name,publisher,positive,negative,total_reviews,positive_ratio
Portal 2,Valve,305671,3770,309441,0.9878167405094994
Stardew Valley,ConcernedApe,497558,9283,506841,0.9816845914201889
Wallpaper Engine,Wallpaper Engine Team,561096,11031,572127,0.9807193158162436
Terraria,Re-Logic,1014711,22380,1037091,0.978420408623737
Left 4 Dead 2,Valve,643836,16828,660664,0.9745286560187932
Euro Truck Simulator 2,SCS Software,572368,15615,587983,0.9734431097497717
Hollow Knight,Team Cherry,246135,7193,253328,0.9716059811785511
Phasmophobia,Kinetic Games,475555,15543,491098,0.968350512524995
Garry's Mod,Valve,861240,29998,891238,0.9663412017889722
The Witcher 3: Wild Hunt,CD PROJEKT RED,632627,25245,657872,0.9616262738040227


Databricks visualization. Run in Databricks to view.

> Les jeux les mieux notés

Il a été nécessaire de filtrer sur les jeux ayant un minimum de votes afin d'éviter les jeux très peu notés avec un ratio de 100% (et non connu du grand public). Une base de 250.000 votes m'est apparu comme raisonnable pour avoir un résultat cohérent. On pourrait bien évidemment revoir ce nombre pour élargir à des jeux moins connus.

Dans l'ensemble, le top 10 des jeux affiche des performances notables avec des ratios de critiques positives très élevés, oscillant entre 96.2 % et 98.8 %. Les différences entre ces ratios sont très minimes, soulignant une satisfaction globale des utilisateurs pour l'ensemble des jeux examinés, y compris des titres populaires tels que Portal 2, Stardew Valley, et Terraria. 

Il est à noter qu'aucun jeu de ce top 10 ne fait partie des éditeurs qui publient le plus sur la plateforme.

### C) Evolution des sorties dans le temps

In [0]:
df_release_year = df.withColumn('year_release', F.substring(F.col('data.release_date'), 1, 4))\
    .groupBy('year_release')\
    .count()\
    .orderBy('year_release', ascending=False)\
    .limit(20)

display(df_release_year)

year_release,count
2022,7455
2021,8823
2020,8305
2019,6968
2018,7678
2017,6017
2016,4185
2015,2576
2014,1557
2013,471


Databricks visualization. Run in Databricks to view.

> Evolution des sorties dans le temps

Depuis sa création, Steam a eu plusieurs phases de sorties de jeux.

A partir de 2003, on constate un premier essort des sorties sur la plateforme, jusqu'à atteindre 471 sorties en 2013.

Les sorties de jeux vidéo se sont grandement intensifiés dès 2014, et augmenté progressivement chaque année avant de se stabiliser en 2018 aux alentours de 7500 sorties.

Le Covid n'a pas eu d'impact négatif sur les sorties, au contraire même on constate une évolution positive de 18.6 % entre 2019 et 2020

### D) Distribution des prix de vente

In [0]:
window_spec = Window.orderBy()

df_price = df.select(F.col('data.price').cast('float').alias('price'))\
    .withColumn('price', F.round(F.col('price')/100))\
    .filter(F.col('price') <= 60)\
    .groupBy('price')\
    .count()\
    .orderBy('price')\
    .withColumn('price_distribution', F.col('count') / F.sum('count').over(window_spec))

display(df_price)

price,count,price_distribution
0.0,8168,0.14698842880023033
1.0,5856,0.10538249743562057
2.0,4492,0.08083643758210513
3.0,3779,0.0680055426586766
4.0,2678,0.04819233745433605
5.0,6340,0.1140923896417067
6.0,1919,0.03453364285842826
7.0,1931,0.03474959059907502
8.0,1614,0.029044971116989688
9.0,768,0.013820655401392863


Databricks visualization. Run in Databricks to view.

> Distribution du prix des jeux sur STEAM

On peut déduire plusieurs paliers de prix sur steam :
1) Les jeux gratuits <b>(15%)</b>
2) Les jeux de 1 à 4 € <b>(30%)</b>
3) Les jeux entre 5 et 9 € <b>(23%)</b>
4) Les jeux à 10 et 14 € <b>(15%)</b>
5) Les jeux supérieurs à 15€, par palier de 5€ <b>(17%)</b>

Steam est une plateforme où plus de 80% des jeux qui se vendent sont à moins de 15€

### E) Les jeux en promotion

In [0]:
df_discount = df.select(F.col('data.discount').alias('discount'))\
    .filter(F.col('discount') > 0)\
    .count()

print(f'Nombre de jeux en promo : {df_discount}')
print(f'Nombre de jeux total : {df_count}')
print(f'Ratio promo/total : {df_discount/df_count*100:.2f}%')

Nombre de jeux en promo : 2518
Nombre de jeux total : 55691
Ratio promo/total : 4.52%


> Les jeux en promotion

On retrouve peu de jeux en promotion sur la plateforme, seulement 4.52 %. 

Il aurait été intéressant de voir le même jeu de données sur d'autres périodes (plus ou moins commerciales) pour estimer la tendance des promotions et les ventes associées.

### F) Les langues les plus représentés

In [0]:
df_languages = df.groupBy('data.languages')\
    .count()\
    .withColumn('languages', F.explode(F.split(F.col('languages'), ', ')))\
    .groupBy('languages')\
    .sum('count')\
    .withColumnRenamed('sum(count)', 'count')\
    .sort('count', ascending=False)\
    .limit(10)\
    .withColumn('language_distribution', F.col('count') / df_count)

display(df_languages)

languages,count,language_distribution
English,55116,0.9896751719308327
German,14019,0.251728286437665
French,13426,0.24108024635937583
Russian,12922,0.23203031010396652
Simplified Chinese,12782,0.2295164389219084
Spanish - Spain,12233,0.21965847264369467
Japanese,10368,0.18617011725413443
Italian,9304,0.16706469627049253
Portuguese - Brazil,6750,0.12120450342066043
Korean,6599,0.11849311378858343


Databricks visualization. Run in Databricks to view.

In [0]:
language_english = df_languages.select('language_distribution').collect()[0][0]

print(f'Ratio jeux en anglais / nombre de jeux sur Steam = {language_english * 100:.2f} %')

Ratio jeux en anglais / nombre de jeux sur Steam = 98.97 %


> Les langues les plus représentées

L'anglais est incontestablement le langage le plus représenté sur Steam, présent dans quasiment 99 % des jeux. Il semble donc crucial, lors de la création d'un jeu vidéo, de le rendre accessible en anglais tant au niveau du texte que des sons.

D'autres langues sont également bien représentées, comme l'allemand, le français, le chinois, l'espagnol etc.

### G) Les jeux réservés aux personnes de 16 ans et plus

In [0]:
df_required_age = df.filter(F.col('data.required_age').cast('int') >= 16)\
    .groupBy('data.required_age')\
    .count()\
    .sort('count', ascending=False)\
    .limit(3)

required_age_sum = df_required_age.agg({'count': 'sum'}).collect()[0][0]

print(f'{required_age_sum} jeux interdits au moins de 16 ans, {required_age_sum / df_count * 100:.2f} % des jeux de la plateforme')

299 jeux interdits au moins de 16 ans, 0.54 % des jeux de la plateforme


> Les jeux réservés aux personnes de 16 ans et plus

Au global, on voit que la plateforme ne possède que 299 jeux interdits au moins de 16 ans. Cela représente la faible proportion de 0.54 %. 

Ce n'est pas un élément qui sera très important dans notre analyse.

## PARTIE 2 : ANALYSE PAR CATEGORIE

### A) Les catégories les plus représentées

In [0]:
df_genres = df.groupBy('data.genre')\
    .count()\
    .withColumn('genre', F.explode(F.split(F.col('genre'), ', ')))\
    .groupBy('genre')\
    .sum('count')\
    .withColumnRenamed('sum(count)', 'count')\
    .sort('count', ascending=False)\
    .limit(10)\
    .withColumn('genre_distribution', F.col('count') / df_count)

display(df_genres)

genre,count,genre_distribution
Indie,39681,0.7125208741089224
Action,23759,0.4266218958179957
Casual,22086,0.3965811351924009
Adventure,21431,0.3848198093049146
Strategy,10895,0.19563304663231043
Simulation,10836,0.19457362949130022
RPG,9534,0.17119462749815947
Early Access,6145,0.11034098866962345
Free to Play,3393,0.060925463719451976
Sports,2666,0.047871289795478625


Databricks visualization. Run in Databricks to view.

> Quels sont les genres les plus représentés ?

On voit que les jeux indépendants sont les jeux les plus représentés avec un taux de 71.25 %. Un résultat qui semble logique quand on compare avec le fait que plus de 80% des jeux sont à un prix de moins de 15 €

Les jeux d'actions, grand public (casual) et d'aventures sont également bien représentés à plus de 38% chacun.

### B) Les catégories les mieux notés

In [0]:
# Filtre sur le nombre de votes >= 1.000.000

df_genres_ratio = df.select('data.genre', 'data.positive', 'data.negative')\
    .withColumn('genre', F.explode(F.split(F.col('genre'), ', ')))\
    .groupBy('genre')\
    .agg(
        F.sum(F.col('positive')).alias('positive'),
        F.sum(F.col('negative')).alias('negative')
    )\
    .withColumn('total_reviews', F.col('positive') + F.col('negative'))\
    .withColumn('positive_ratio', F.col('positive') / F.col('total_reviews'))\
    .filter(F.col('genre') != '')\
    .filter(F.col('total_reviews') >= 1000000)\
    .orderBy('positive_ratio', ascending=False)\
    .limit(10)

display(df_genres_ratio)

genre,positive,negative,total_reviews,positive_ratio
Indie,32531023,4241234,36772257,0.8846621244923857
Casual,10034967,1537296,11572263,0.8671568387272222
Simulation,15572390,2400512,17972902,0.8664371507728691
Racing,2340353,383691,2724044,0.8591465482936399
RPG,19425528,3274328,22699856,0.855755560740121
Action,54858618,9687659,64546277,0.8499114209174295
Strategy,13402870,2393425,15796295,0.8484818750219593
Adventure,29689445,5653153,35342598,0.8400470446456709
Early Access,4334595,936191,5270786,0.8223811401183808
Free to Play,18722246,4280807,23003053,0.8139026589209701


Databricks visualization. Run in Databricks to view.

> Quels sont les genres avec le meilleur ratio d'avis positif/négatif ?

Les ratios d'avis positif/negatif sont assez proches les uns des autres selon les genres. Les jeux indépendants sont très légèrement en tête, suivi des jeux grand publics et des simulations. 

Un filtre a volontairement été appliqué pour n'afficher que les genres qui ont eu un minimum d'1 million d'avis, pour éviter d'avoir des catégories de niches avec très peu d'avis.

### C) Les catégories préférées des éditeurs

In [0]:

df_genres_publisher = df.select('data.publisher', 'data.genre')\
  .withColumn('genre', F.explode(F.split(F.col('genre'), ', ')))\
  .groupBy('publisher', 'genre')\
  .count()\
  .filter(F.col('publisher') != '')\
  .filter(F.col('genre') != '')\
  .orderBy('count', ascending=False)\

top_10_publishers = df_publisher.select('publisher')

df_genres_publisher = df_genres_publisher.join(top_10_publishers, 'publisher', 'inner')\
  .filter(F.col('count') >= 10)

display(df_genres_publisher)

publisher,genre,count
HH-Games,Indie,69
Ubisoft,Racing,14
8floor,Strategy,22
HH-Games,Adventure,39
Choice of Games,Casual,28
Big Fish Games,Casual,418
Ubisoft,Strategy,22
HH-Games,Simulation,12
Strategy First,Simulation,25
Square Enix,Strategy,13


Databricks visualization. Run in Databricks to view.

> Les catégories préférées des éditeurs

Les éditeurs sont plus ou moins concentrés sur des catégories. Les concurrents 8floor et Big Fish Games sont eux présents sur peu de catégories (principalement casual et aventure).

Ubisoft se démarque en étant présents dans diverses catégories de jeux :
- Action
- Aventure
- Stratégie
- RPG
- Simulation
- Racing
- Casual

Les éditeurs qui se rapprochent le plus d'Ubisoft sont Square Enix et SEGA, probablement des concurrents de taille.

Ubisoft est présent dans les catégories les plus représentées sur Steam, et n'est pas du tout présent dans les logiciels.

### D) Les catégories les plus rentables

In [0]:
df_profit = df.select('data.genre', 'data.price', 'data.owners')\
    .withColumn('genre', F.explode(F.split(F.col('genre'), ', ')))\
    .withColumn('price', F.round(F.col('price')/100))\
    .withColumn('lower', F.trim(F.regexp_replace(F.split(F.col('owners'), '\.\.')[0], ',', '')))\
    .withColumn('upper', F.trim(F.regexp_replace(F.split(F.col('owners'), '\.\.')[1], ',', '')))\
    .withColumn('average_owners', (F.col('lower') + F.col('upper')) / 2)\
    .withColumn('sales', F.col('average_owners') * F.col('price'))\
    .groupBy('genre')\
    .agg(
        F.count(F.col('sales')).alias('count'),
        F.sum(F.col('sales')).alias('sales'),
        F.sum(F.col('average_owners')).alias('average_owners')
    )\
    .orderBy('sales', ascending=False)\
    .limit(10)

display(df_profit)

genre,count,sales,average_owners
Action,23759,5.8769775E10,4.77781E9
Adventure,21431,3.726015E10,2.65988E9
Indie,39681,3.2358145E10,3.18869E9
RPG,9534,2.7178705E10,1.746625E9
Strategy,10895,2.015419E10,1.636135E9
Simulation,10836,1.877436E10,1.3336E9
Casual,22086,8.09035E9,1.283345E9
Massively Multiplayer,1460,5.932245E9,9.6533E8
Early Access,6145,5.46148E9,4.3593E8
Sports,2666,3.150745E9,3.2007E8


Databricks visualization. Run in Databricks to view.

> Les catégories les plus rentables

En réalité nous ne pouvons pas nous attarder sur la rentabilité (car nous ne connaissons pas les dépenses, ni le chiffre d'affaires exact). Cependant nous allons nous baser sur un produit entre le prix, et le nombre moyen de personne possédant le jeu.

En explorant les données des ventes de jeux par catégorie, certaines tendances émergent. Voici quelques points intéressants :

- Action domine en termes de ventes totales, surpassant toutes les autres catégories avec un chiffre impressionnant de 58 M. Cela pourrait également être dû à une base d'utilisateurs plus importante.

- Adventure, Indie et RPG suivent de près, démontrant une rentabilité notable avec respectivement 37 M , 32 M et 27M en ventes totales.

- Strategy et Simulation, bien que légèrement inférieures en ventes totales, mettent tout de même en lumière une base d'utilisateurs fidèles et engagés.

Cette analyse permet d'identifier des nuances dans la rentabilité des catégories de jeux, offrant des insights pour les éditeurs et développeurs souhaitant maximiser leur impact financier sur la plateforme.

## PARTIE 3 : ANALYSE PAR PLATEFORME

### A) Les disponibilités des jeux selon les systèmes d'exploitation

In [0]:
df_platforms = df.select("data.platforms.linux", "data.platforms.mac", "data.platforms.windows")\
    .withColumn("linux", F.col("linux").cast("int"))\
    .withColumn("mac", F.col("mac").cast("int"))\
    .withColumn("windows", F.col("windows").cast("int"))\
    .agg(
        F.sum("linux").alias("linux_sum"),
        F.sum("mac").alias("mac_sum"),
        F.sum("windows").alias("windows_sum"))\
    .selectExpr(
        "stack(3, 'linux', linux_sum, 'mac', mac_sum, 'windows', windows_sum) as (platform, sum)"
    )\
    .withColumn("ratio", F.col("sum") / df_count)\
    
display(df_platforms)

platform,sum,ratio
linux,8458,0.15187373184176978
mac,12770,0.22930096424916055
windows,55676,0.9997306566590652


Databricks visualization. Run in Databricks to view.

> Les disponibilités des jeux selon les systèmes d'exploitation

Steam, principalement conçu pour Windows, assure une compatibilité quasiment complète avec le système de Microsoft, permettant ainsi de jouer à presque 100% des jeux disponibles (et également d'utilisation des logiciels)

Malheureusement, la représentation des compatibilités avec Mac et Linux est nettement moindre, se situant respectivement à 23% et 15%.

### B) L'influence des catégories sur le choix des plateformes

In [0]:
df_platforms_categories = df.select('data.genre', 'data.platforms.linux', 'data.platforms.mac', 'data.platforms.windows')\
  .withColumn('genre', F.explode(F.split(F.col('genre'), ', ')))\
  .filter(~(F.col("genre").contains("Movie") | (F.col("genre") == "")))\
  .withColumn("linux", F.col("linux").cast("int"))\
  .withColumn("mac", F.col("mac").cast("int"))\
  .withColumn("windows", F.col("windows").cast("int"))\
  .groupBy('genre')\
  .agg(
    (F.sum("linux") / F.count("linux")).alias('linux_ratio'),
    (F.sum("mac") / F.count("mac")).alias("mac_ratio"),
    (F.sum("windows") / F.count("windows")).alias("windows_ratio"))

In [0]:
df_platforms_linux = df_platforms_categories.select('genre', 'linux_ratio')\
    .sort('linux_ratio', ascending=False)

display(df_platforms_linux)

genre,linux_ratio
Game Development,0.22012578616352202
Indie,0.17585242307401527
Strategy,0.16759981642955485
RPG,0.15984896161107615
Nudity,0.15555555555555556
Adventure,0.15407587140124118
Casual,0.14964230734401884
Action,0.14221979039521865
Gore,0.1414141414141414
Simulation,0.14138058324104835


Databricks visualization. Run in Databricks to view.

In [0]:
df_platforms_mac = df_platforms_categories.select('genre', 'mac_ratio')\
    .sort('mac_ratio', ascending=False)

display(df_platforms_mac)

genre,mac_ratio
Game Development,0.3270440251572327
Strategy,0.2758145938503901
Indie,0.2503717144225196
Accounting,0.25
Free to Play,0.24904214559386972
Design & Illustration,0.24630541871921183
Sexual Content,0.24074074074074073
RPG,0.23578770715334593
Adventure,0.2351266856422939
Casual,0.23227383863080683


Databricks visualization. Run in Databricks to view.

> L'influence des catégories sur le choix des plateformes

Nous allons ici nous concentrer sur Linux et Mac. Sur le système Windows, toutes les catégories sont presques entièrement disponibles, il n'y a pas d'intérêt à l'analyser.

Pour les deux systèmes d'exploitation, on retrouve en tête les logiciels de développement de jeux vidéo, suivi par les jeux indépendants et les jeux de stratégies. Les autres catégories sont également assez proche du trio de tête.

On remarque également que certains logiciels ne sont pas du tout disponibles sur Linux et peu présents sur Mac, cependant Ubisoft n'est pas concerné par cela.

Pour savoir s'il est important de développer des jeux compatibles avec Linux et Mac, il serait utile d'obtenir les chiffres d'utilisation/téléchargement de chaque jeu avec les systèmes d'exploitation.